# ETL_EDA

ETL: Extract, Transform, Load \
EDA: Exploratory Data Analysis

The World Bank's World Development Indicator datasets were already uploaded to Google Cloud Platform's BigQuery interface. THE ETL process was to simply connect the GCP BigQuery interface to the application.

## 1. Schema
There are six datasets(tables) in world_bank_wdi folder from Google Cloud's BigQuery public datasets. The tables are as follows:
- country_series_definitions
- country_summary
- footnotes
- indicators_data
- series_summary
- series_time

The main dataset is the indicators_data. For various kinds of indicators, each row is the indicator's value at a certain year for a certain country. The schema of the table is as follows:
- country_name
- country_code
- indicator_name
- indicator_code
- value
- year

For all distinct indicators, I saved a quick sql query of the indicators (simple SELECT DISTINCT call) into a google sheet and saved it as indicators in the Google Drive folder (I have also saved the result as a csv file to the github repo).

The other five tables are supporting datasets. country_series_definition, series_time, footnotes are actually definitions and supporting explanations about the dataset. country_summary and series_summary however contains information that are relevant to each country and each indicator. I have also saved all schemas for all tables as Schema google sheet (also in Github repo).

## 2. Indicators
There are total 1431 unique indicators. However, it is important to use indicators that are not 'null' to some countries. Below is a simple BigQuery query to extract the number of indicators for all countries.

The saved result is called num_indicators.csv. I realized that the number of indicators are significantly lacking for many countries. The primary reason for this seemed to be because they were very small island countries or city states (Monaco, Virgin Islands), or data access was virtually impossible (North Korea). Although I won't be able to use these countries, I chose to use the indicators that the country named 'World' had. Below was a code to extract the main indicators for this project: 

This country_code seemed to be containing the average values of the indicators of all the countries for each year. Some representative indicators from this country from the head of the dataset are as follows:
- Access to clean fuels and technologies for cooking (% of population)
- Access to electricity (% of population)
- CO2 emissions (kt)

## 3. Feature Engineering (Indicators)
Now we narrowed the indicators down to approximately 907, we acutally perform some EDA to look into what kind of indicators there are. 

There were total 11 large themes, such as Economics, Environment, Health, Infrastrcuture, etc. The dataset however were marked with subtopics, such as in the format of Environment: Energy. The first step was to merge these into big themes to simplify the process. To list all the topics, they are as follows:
- Economic Policy & Debt
- Education
- Environment
- Financial Sector
- Gender
- Health
- Infrastructure
- Poverty
- Private Sector & Trade
- Public Sector
- Social Protection & Labor

Then, among 907 indicators, I was able to identify very similar indicators. Some were the same indicators with different units, some were the same ones with two different measurements for female and male (with combined as a separate indicator), or some were a lot more sub-categorized, such as electricity production from various kinds of renewable energy. Considering the sheer amount of indicators from the whole dataset, I simply meddled with the string value of the indicator name to only get the common, and thus representative indicators for these various ones. By doing so, I was able to reduce the indicators to 453. The following is the query job for this process.

After reducing down the indicators, I analyzed them through and found that it would be possible to pick a representative indicator for each topic, and can create a chart for each of the 11 indicators to the website, possibly with toggling functions enabled.

The other ones then could be used as numerous features to be run through clusering algorithms for machine learning enhancements.

## 4. Data Transfer
Now, the most important part of this project is to send these organized data to the website, by country. Thus, the website would need a function to choose a country, and its time series data should be uploaded from the SQL server and be ready to be visualized.

Thus, the following query is used for the world average data.

In [8]:
import pandas

sql = """
SELECT indicator, year, value
FROM `worlddev.wdi.main`
WHERE country_code = 'WLD'
LIMIT 100
"""

project_id = 'worlddev'

df = pandas.read_gbq(sql, project_id=project_id, dialect='standard')
df

Please visit this URL to authorize this application: https://accounts.google.com/o/oauth2/auth?response_type=code&client_id=725825577420-unm2gnkiprugilg743tkbig250f4sfsj.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fbigquery&state=ju8kvpnZoexQybPS0rYht7PfEQeWck&prompt=consent&access_type=offline
Enter the authorization code: 4/uQF-mMewSkrPDKuB5qmDQdEYExRlDwLk8NDqanBYQciueeFH_wGjo0g


Downloading: 100%|██████████| 100/100 [00:00<00:00, 296.42rows/s]


,indicator,year,value
0,Access to clean fuels and technologies for coo...,2000,4.940514e+01
1,Access to clean fuels and technologies for coo...,2001,5.021034e+01
2,Access to clean fuels and technologies for coo...,2002,5.081952e+01
3,Access to clean fuels and technologies for coo...,2003,5.141249e+01
4,Access to clean fuels and technologies for coo...,2004,5.206288e+01
...,...,...,...
95,Adjusted net national income,2009,5.154397e+13
96,Adjusted net national income,2010,5.403049e+13
97,Adjusted net national income,2011,5.560511e+13
98,Adjusted net national income,2012,5.732322e+13


We can then parametrize this to find certain country's certain indicator time series data.

In [10]:
sql = """
SELECT country_name, indicator, year, value
FROM `worlddev.wdi.main`
WHERE country_code = @country_code AND indicator = @indicator
"""

def config(country_code, indicator):
    query_config = {
        'query': {
            'parameterMode': 'NAMED',
            'queryParameters': [
                {
                    'name': 'country_code',
                    'parameterType': {'type': 'STRING'},
                    'parameterValue': {'value': country_code}
                },
                {
                    'name': 'indicator',
                    'parameterType': {'type': 'STRING'},
                    'parameterValue': {'value': indicator}
                }
            ]
        }
    }
    return query_config

# Run a Standard SQL query with the project set explicitly
project_id = 'worlddev'
country_code = 'WLD'
indicator = 'Access to clean fuels and technologies for cooking'
df = pandas.read_gbq(sql, configuration=config(country_code, indicator), project_id=project_id)
df

Downloading: 100%|██████████| 17/17 [00:00<00:00, 47.97rows/s]


,country_name,indicator,year,value
0,World,Access to clean fuels and technologies for coo...,2000,49.405144
1,World,Access to clean fuels and technologies for coo...,2001,50.210340
2,World,Access to clean fuels and technologies for coo...,2002,50.819520
3,World,Access to clean fuels and technologies for coo...,2003,51.412494
4,World,Access to clean fuels and technologies for coo...,2004,52.062882
5,World,Access to clean fuels and technologies for coo...,2005,52.674512
6,World,Access to clean fuels and technologies for coo...,2006,53.420248
7,World,Access to clean fuels and technologies for coo...,2007,54.065305
8,World,Access to clean fuels and technologies for coo...,2008,54.757178
9,World,Access to clean fuels and technologies for coo...,2009,55.327743


Using this simple query, I will visualize various kinds of indicators for different countries. The visualization codes will be in Visualization.ipynb. I will also bring in many indicator queries at the same time to conduct clustering and knn/kmeans analysis as well.